In [ ]:
pip install langchain transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 756.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoProcessor, AutoModel

# kobert
kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
kobert_model = AutoModel.from_pretrained("skt/kobert-base-v1")

# koclip
koclip_processor = AutoProcessor.from_pretrained("koclip/koclip-base-pt")
koclip_model = AutoModel.from_pretrained("koclip/koclip-base-pt")

preprocessor_config.json:   0%|          | 0.00/521 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.70G [00:00<?, ?B/s]

In [ ]:
# 1. LLM Chain
# 언어 모델(LLM)을 기반으로 한 체인입니다. 주로 텍스트 생성, 대화, 번역 등에 사용됩니다.
from langchain.llms import HuggingFaceLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = HuggingFaceLLM(model=model, tokenizer=tokenizer)
prompt_template = PromptTemplate(template="Translate to French: {text}", input_variables=["text"])
chain = LLMChain(llm=llm, prompt=prompt_template)


# 2. Sequential Chain
# 여러 체인을 순차적으로 실행하는 체인입니다. 한 체인의 출력이 다음 체인의 입력으로 사용됩니다.
from langchain.chains import SequentialChain

chain1 = LLMChain(llm=llm, prompt=PromptTemplate(template="Summarize: {text}", input_variables=["text"]))
chain2 = LLMChain(llm=llm, prompt=PromptTemplate(template="Translate to French: {summary}", input_variables=["summary"]))

sequential_chain = SequentialChain(chains=[chain1, chain2], input_variables=["text"], output_variables=["summary", "translation"])


# 3. Parallel Chain
# 여러 체인을 병렬로 실행하는 체인입니다. 각 체인은 동일한 입력을 받아 독립적으로 실행됩니다.
from langchain.chains import ParallelChain

chain1 = LLMChain(llm=llm, prompt=PromptTemplate(template="Summarize: {text}", input_variables=["text"]))
chain2 = LLMChain(llm=llm, prompt=PromptTemplate(template="Generate questions: {text}", input_variables=["text"]))

parallel_chain = ParallelChain(chains=[chain1, chain2], input_variables=["text"], output_variables=["summary", "questions"])


# 4. Router Chain
# 입력에 따라 적절한 체인을 선택하여 실행하는 체인입니다.
from langchain.chains import RouterChain

chain1 = LLMChain(llm=llm, prompt=PromptTemplate(template="Summarize: {text}", input_variables=["text"]))
chain2 = LLMChain(llm=llm, prompt=PromptTemplate(template="Translate to French: {text}", input_variables=["text"]))

def router_function(inputs):
    if 'translate' in inputs['task']:
        return chain2
    else:
        return chain1

router_chain = RouterChain(router_function=router_function, input_variables=["text", "task"])


# 5. 함수 기반 체인
# 함수를 체인으로 연결하여 작업을 수행하는 예시입니다.
from langchain.chains import SimpleChain

# Define individual functions
def text_preprocessing(text):
    return text.lower().strip()

def summarize(text):
    return "Summary of: " + text

def translate_to_french(text):
    return "Traduction: " + text

# Create chains for each function
preprocess_chain = SimpleChain(lambda inputs: {'text': text_preprocessing(inputs['text'])})
summarize_chain = SimpleChain(lambda inputs: {'summary': summarize(inputs['text'])})
translate_chain = SimpleChain(lambda inputs: {'translation': translate_to_french(inputs['summary'])})

# Sequentially chain the functions
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[preprocess_chain, summarize_chain, translate_chain],
                        input_variables=["text"],
                        output_variables=["translation"])

# Input text
text = "  This is a SAMPLE text for Testing.  "
result = chain.run({"text": text})
print(result)  # Output will include the translation


# 6. 클래스 기반 체인
# 클래스를 체인으로 연결하는 예시입니다.
class TextPreprocessor:
    def process(self, text):
        return text.lower().strip()

class Summarizer:
    def summarize(self, text):
        return "Summary of: " + text

class Translator:
    def translate(self, text):
        return "Traduction: " + text

# Instances of classes
preprocessor = TextPreprocessor()
summarizer = Summarizer()
translator = Translator()

# Define the chains
preprocess_chain = SimpleChain(lambda inputs: {'text': preprocessor.process(inputs['text'])})
summarize_chain = SimpleChain(lambda inputs: {'summary': summarizer.summarize(inputs['text'])})
translate_chain = SimpleChain(lambda inputs: {'translation': translator.translate(inputs['summary'])})

# Sequentially chain the class methods
chain = SequentialChain(chains=[preprocess_chain, summarize_chain, translate_chain],
                        input_variables=["text"],
                        output_variables=["translation"])

# Input text
text = "  This is a SAMPLE text for Testing.  "
result = chain.run({"text": text})
print(result)  # Output will include the translation


# 7. 사용자 정의 체인
# 여러 기능을 포함한 사용자 정의 체인을 만들 수 있습니다.
from langchain.chains import Chain

class CustomChain(Chain):
    def __init__(self, preprocessor, summarizer, translator):
        self.preprocessor = preprocessor
        self.summarizer = summarizer
        self.translator = translator

    def _call(self, inputs):
        text = self.preprocessor.process(inputs['text'])
        summary = self.summarizer.summarize(text)
        translation = self.translator.translate(summary)
        return {'translation': translation}

# Instances of classes
preprocessor = TextPreprocessor()
summarizer = Summarizer()
translator = Translator()

# Create and run the custom chain
custom_chain = CustomChain(preprocessor, summarizer, translator)
result = custom_chain({'text': "  This is a SAMPLE text for Testing.  "})
print(result)  # Output will include the translation